In [39]:
%%capture
import pandas as pd
import requests as rq
import pandas as pd
import pandas_profiling
import calendar as cl
import os
import time
import json
import re
import numpy as np
import glob
import sqlite3
from IPython.display import display


In [40]:
#generamos el rango de fechas para realizar la busqueda

Dates = pd.date_range(start='2022-12-01', end='2022-12-31', freq='D', normalize=False)

Dates_Series = Dates.to_series().dt.strftime('%Y-%m-%d')

In [41]:
### Scafold project 

try:
  os.makedirs("../json/Raw_period_date=2022-12")
except:
  pass


try:
  os.makedirs("profiling")
except:
  pass


try:
  os.makedirs("../data/Raw_parquet_period=2022-12")
except:
  pass

try:
  os.makedirs("../data/master_parquet_period=2022-12")
except:
  pass

try:
  os.makedirs("../db/")
except:
  pass




In [ ]:
#@title Data handdler
#data handdler


# Define the API endpoint
index_date = 0

for date_find in Dates_Series :
  # print(date_find)


  response = rq.get(f"http://api.tvmaze.com/schedule/web?date={date_find}")

  
  # valida el status
  if response.status_code == 200:
      # The request was successful, so parse the response data
      data = json.loads(response.content)
      # Save the data to a JSON file
      with open(f"../json/Raw_period_date=2022-12/{Dates_Series[date_find]}.json", "w") as f:
          json.dump(data, f, indent=4)
          f.close()
          time.sleep(0.10)
  else:
      # The request failed, so print the error message
      print(response.status_code)
      print(response.content)




In [ ]:
%%capture 
#data profilling

def return_profilling_per_date(day):


    df = pd.read_json(f"../json/Raw_period_date=2022-12/{day}.json")
    profiling = df.profile_report(title=f'Profiling {day}')
    profiling.to_file(f'../profiling/profiling_{day}.html')

    return "ok"

for date_find in Dates_Series :
    return_profilling_per_date(date_find)
    

In [42]:
#limpieza para columna rating 0 para none 

def clean_rating(df,column):
    
   df[column] = df[column].apply(lambda x: x.get("average"))

   df[column] = df[column].replace(np.NAN, 0)
    
   return df
    



In [43]:
#limpieza de summary elimanar etiquetas ej: <p>
def clean_summary(df, column):

    def remove_tags(text):
        """Removes any type of tag from text."""
        pattern = re.compile(r"<[^>]+>", re.DOTALL)
        return pattern.sub(r"", text)

    df[column] = df[column].apply(lambda x: remove_tags(str(x)))
    return df

    

In [5]:
%%capture
## descomponemos la columna _embedded la cual contiene informacion embebida relevante 
values_id_show = list()
def overturn_column_show_embedded_columns(df, key):
    values_id_show.clear()
    for index_row in range(0,len(df.index)):
        values_id_show.append(df["_embedded"][index_row]["show"][key])
    return values_id_show

In [44]:
#lista de cloumnas embebidas 
list_of_keys = [
  "id",
  "url",
  "name",
  "type",
  "language",
  "genres",
  "status",
  "runtime",
  "averageRuntime",
  "premiered",
  "ended",
  "officialSite",
  "schedule",
  "rating",
  "weight",
  "network",
  "webChannel",
  "dvdCountry",
  "externals",
  "image",
  "summary",
  "updated",
  "_links",]

def create_columns_ebedded(df):
    for key in list_of_keys:
        df[f"show_{key}"]=overturn_column_show_embedded_columns(df, key)
    return df

In [8]:
list_columns_object = [
       'url', 'name', 'type','show_url', 'show_type','image','airtime',
       'summary', 'show_name','show_language', 'show_genres', 
       'show_status', 'show_premiered', 'show_ended', 'show_officialSite', 
       'show_schedule', 'show_network','show_webChannel', 'show_dvdCountry', 
       'show_externals', 'show_image','show_summary', 'show__links']

def cast_object_to_string(df):
        for column in list_columns_object:
            df[column] = df[column].astype("string")

        return df

In [17]:
list_of_colums_date = ["airdate"]

def cast_object_to_date(df):
    for column in list_of_colums_date:
        df[column] = pd.to_datetime(df[column])

In [45]:
#captura de data fecha por fecha 

def wranglind_data(date):

    df_day = pd.read_json(f"../json/Raw_period_date=2022-12/{date}.json")
    create_columns_ebedded(df_day)
    clean_summary(df_day,'show_summary')
    clean_summary(df_day,'summary')
    clean_rating(df_day,"rating")
    clean_rating(df_day,"show_rating")
    df_day = df_day.drop(['_embedded'], axis=1)
    df_day = df_day.drop(['_links'], axis=1)
    cast_object_to_string(df_day)
    #cast_object_to_date(df_day)


    return df_day


    
for day_to_wranglind_data in Dates_Series:
    data_warngled=wranglind_data(day_to_wranglind_data)
    data_warngled.to_parquet(f"../data/master_parquet_period=2022-12/{day_to_wranglind_data}.snappy.parquet")
    

# lectura de parquets 



In [46]:
## muestra de lectrua de archivo parquet
df_parquet = pd.read_parquet("../data/master_parquet_period=2022-12/2022-12-31.snappy.parquet")

In [47]:
## buesqueda recursiva de parquets sobre la carpeta json
import pandas as pd

file_data_parquet = "../data/master_parquet_period=2022-12/*.parquet"
files = glob.glob(file_data_parquet, recursive=False)


In [48]:
#concatenacion de los n parquets en un solo dataframe
df = pd.concat([pd.read_parquet(file) for file in files], ignore_index=True)

In [49]:

# Conteo de shows de tv por género
generos_show = df.groupby([df["show_type"]]).count()
generos_show["show_name"]

show_type
Animation       520
Award Show        2
Documentary     260
Game Show       104
News             23
Reality         499
Scripted       2214
Sports           91
Talk Show       195
Variety          76
Name: show_name, dtype: int64

In [19]:
## runtime promedio por show
df["runtime"].groupby([df["show_name"], df["show_id"]]).mean()

show_name           show_id
#BOOS               40471      25.0
1 For All           62858       7.0
10 глупых вопросов  40482      23.0
110                 62979      40.0
13 клиническая      64517      43.0
                               ... 
Я СЕБЯ ЗНАЮ!        47865      67.0
Я слежу за тобой    65481       NaN
Я тебе не верю      59120      45.0
вМесте. Интервью    47027      36.0
ебаут               55566      77.2
Name: runtime, Length: 747, dtype: float64

In [15]:
df["show_officialSite"].groupby([df["show_name"]]).unique()
# df.show_officialSite.unique()

show_name
#BOOS                 [https://www.youtube.com/channel/UCc0kHafEIzm6...
1 For All             [https://www.youtube.com/user/deerstalkerpictu...
10 глупых вопросов    [https://www.youtube.com/playlist?list=PLLkqhq...
110                                       [https://tv.nrk.no/serie/110]
13 клиническая           [https://www.ivi.tv/watch/13-ya-klinicheskaya]
                                            ...                        
Я СЕБЯ ЗНАЮ!          [https://www.youtube.com/playlist?list=PLbwnGd...
Я слежу за тобой          [https://premier.one/show/ya-slezhu-za-toboj]
Я тебе не верю                [https://ya-tebe-ne-veryu.tnt-online.ru/]
вМесте. Интервью                    [http://vmesteproject.ru/intervue/]
ебаут                 [https://www.youtube.com/c/%D0%B5%D0%B1%D0%B0%...
Name: show_officialSite, Length: 747, dtype: object

In [50]:
df.to_parquet(f"../data/master_parquet_period=2022-12/master_data_2022-12.snappy.parquet")


In [53]:
#crearmos base de datos y tabla  en sqlite

# Create a connection to the database
conn = sqlite3.connect("../db/tv_episodes.db")

# Create a cursor
cur = conn.cursor()

# Create the table
cur.execute("""
CREATE TABLE IF NOT EXISTS tv_shows (
  id INTEGER PRIMARY KEY NOT NULL,
  url TEXT,
  name TEXT,
  season INTEGER,
  number REAL,
  type TEXT,
  airdate DATETIME,
  airtime TEXT,
  airstamp OBJECT,
  runtime REAL,
  rating REAL,
  image TEXT,
  summary TEXT,
  show_id INTEGER NOT NULL,
  show_url TEXT,
  show_name TEXT,
  show_type TEXT,
  show_language TEXT,
  show_genres TEXT,
  show_status TEXT,
  show_runtime REAL,
  show_averageRuntime REAL,
  show_premiered TEXT,
  show_ended TEXT,
  show_officialSite TEXT,
  show_schedule TEXT,
  show_rating REAL,
  show_weight INTEGER,
  show_network TEXT,
  show_webChannel TEXT,
  show_dvdCountry TEXT,
  show_externals TEXT,
  show_image TEXT,
  show_summary TEXT,
  show_updated INTEGER,
  show__links TEXT
);
""")

# Commit the changes to the database
conn.commit()

# Close the connection to the database
conn.close()

In [59]:


# Importar el archivo Parquet
df = pd.read_parquet("../data/master_parquet_period=2022-12/master_data_2022-12.snappy.parquet")

# Crear una conexión a la base de datos SQLite
conn = sqlite3.connect("../db/tv_episodes.db")


# Conexión a la base de datos
conn = sqlite3.connect("../db/tv_episodes.db")

# Inserta el DataFrame en la tabla
df.to_sql("tv_shows", conn, if_exists="replace")

# Cierra la conexión
conn.close()



In [60]:


# Conexión a la base de datos
conn = sqlite3.connect("../db/tv_episodes.db")

# Obtén la tabla de episodios
df = pd.read_sql("SELECT COUNT(id) FROM tv_shows", conn)

# Imprime el DataFrame
df

,COUNT(id)
0,3984
